Notebook for determing programming status and amount overlap of pcs style data

Importing the necessary libraries

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from pandasql import sqldf

In [2]:
connection_Pavement=("Driver={SQL Server};"
                      "Server=DOTSSMSQLPROD;"                
                      "Trusted_Connection=yes;")
pave_url=URL.create('mssql+pyodbc',query={'odbc_connect':connection_Pavement})
pave_eng=create_engine(pave_url)

In [6]:
def find_uncovered_parts(row):
    #need to add side logic
    rid=row[id1]
    begin_base=row[begin1]
    end_base=row[end1]
    matches=wp_data[wp_data[id2]==rid]
    wpitems=list()
    fy=list()
    uncovered_parts=[(begin_base,end_base)]
    for _,match_row in matches.iterrows():
        begin_second=match_row[begin2]
        end_second=match_row[end2]
        wpitmseg=match_row['WPITEM']+match_row['WPITMSEG']
        fiscal=match_row['FISCALYR']
        new_uncovered_parts=[]
        
        for part_begin,part_end in uncovered_parts:
            #check if parts overlap
            if part_end < begin_second or part_begin > end_second:
                new_uncovered_parts.append((part_begin, part_end))
            else:
                #split the part into uncovered sections
                if part_begin < begin_second:
                    new_uncovered_parts.append((part_begin, begin_second))
                    # wpitems.append(wpitmseg)
                if part_end > end_second:
                    new_uncovered_parts.append((end_second, part_end))
                    # wpitems.append(wpitmseg)
        if uncovered_parts!=new_uncovered_parts:
            wpitems.append(wpitmseg)
            fy.append(fiscal)
        uncovered_parts=new_uncovered_parts
    uncovered_len=sum(end-begin for begin,end in uncovered_parts)
    if len(wpitems)>0:
        wpitems=set(wpitems)
    if len(fy)>0:
        fy_return=min(fy)
    else:
        fy_return='None'
    return uncovered_parts,uncovered_len,wpitems,fy_return

In [7]:
def assoc(bmp,emp,bsec,esec):
    if esec>=emp and bsec<=bmp:
        associated=1
    elif esec>=emp:
        associated=(emp-bsec)/(emp-bmp)
    elif bsec<=bmp:
        associated=(esec-bmp)/(emp-bmp)
    else:
        associated=(esec-bsec)/(emp-bmp)
    if associated>1:
        associated=1
    return associated

In [8]:
def find_programming(pcs_data,wp_data,outpath):
    pcs_data[['uncovered','length_uncovered','list','first_prog']]=pcs_data.apply(find_uncovered_parts,axis=1,result_type='expand')
    pcs_data['percent_uncovered']=pcs_data['length_uncovered']/(pcs_data[end1]-pcs_data[begin1])
    pcs_data.to_excel(Path(outpath,'progamming_added_2.xlsx'),index=False)
    return pcs_data
        
        

In [9]:
def sql_read(type,eng):
    if type=='pcs':
        data=pd.read_sql('Select * from pavement.dbo.master_pcs_net where surveyyear='+str(yr),eng)
    elif type=='wp':
        data=pd.read_sql('Select * from pavemgmt.dbo.wp_fut_ind',eng)
    return data

Identify the names for ID(RDWYID), Begin(BMP), and End(EMP) for pcs data and wp or secondary data.

In [10]:
id1='RDWYID'
begin1='BMP'
end1='EMP'
id2='RDWYID'
begin2='BEGSECPT'
end2='ENDSECPT'

In [11]:
id1=str.upper(id1)
begin1=str.upper(begin1)
end1=str.upper(end1)
id2=str.upper(id2)
begin2=str.upper(begin2)
end2=str.upper(end2)

If pcs_data is in an excel file enter the path below, otherwise it will default to the last completed surveyyear in sql

In [12]:
pcs_directory=r'Q:\Road Survey Data\PCS Network\2024-2025\Distresses\Patching\\' 
pcs_file='PtSections.xlsx'
pcs_path=Path(pcs_directory,pcs_file)

In [13]:
try:
    pcs_data=pd.read_excel(pcs_path)
except:
    pcs_data=sql_read('pcs',pave_eng)
wp_data=sql_read('wp',pave_eng)



In [ ]:
pcs_data.columns=pcs_data.columns.map(str.upper)
wp_data.columns=wp_data.columns.map(str.upper)
wp_data[id2]=wp_data[id2].astype(str)
pcs_data[id1]=pcs_data[id1].astype(str)


In [ ]:
wp_data=wp_data[wp_data['WPSTATUS']!='CA']

In [ ]:
final_data=find_programming(pcs_data,wp_data,pcs_directory)
